In [ ]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

avversari  = pd.read_csv('Avversari.csv')
convocati  = pd.read_csv('Convocati.csv')
eventi     = pd.read_csv('Eventi.csv')
giocatori  = pd.read_csv('Giocatori.csv')
partite    = pd.read_csv('Partite.csv')
tipoeventi = pd.read_csv('TipoEventi.csv')
partite

In [ ]:
EVENT_NAME = {
     0: 'T1ok',
     1: 'T1err',
     2: 'T2ok',
     3: 'T2err',
     4: 'T3ok',
     5: 'T3err',
     6: 'ROff',
     7: 'RDif',
     8: 'PRec',
     9: 'PPer',
    10: 'Ass',
    11: 'SDat',
    12: 'SSub',
    13: 'FCom',
    14: 'FTec',
    15: 'FSub',
    16: 'Espu',
    17: 'Info',
    18: 'Entr',
    19: 'Usci',
    20: 'Time'
}

# Long description of event types
EVENT_DESCRIPTION = {
     0: 'Tiro libero centrato',
     1: 'Tiro libero sbagliato',
     2: 'Tiro da 2 centrato',
     3: 'Tiro da 2 sbagliato',
     4: 'Tiro da 3 centrato',
     5: 'Tiro da 3 sbagliato',
     6: 'Rimbalzo offensivo',
     7: 'Rimbalzo difensivo',
     8: 'Palla recuperata',
     9: 'Palla Persa',
    10: 'Assist',
    11: 'Stoppata data',
    12: 'Stoppata subita',
    13: 'Fallo commesso',
    14: 'Fallo tecnico',
    15: 'Fallo subito',
    16: 'Espulsione',
    17: 'Infortunio',
    18: 'Entrata in campo',
    19: 'Uscita dal campo',
    20: 'Time-out'
}

In [ ]:
import json

for index,row in partite.iterrows():

    gameid = int(row['ID'])
    
    if gameid <= 13:
        continue
    
    # Game general info
    game = {
        "date": row['Data'],
        "time": "18:00",
        "season": "Stagione " + row['Stagione'],
        "championship": "Campionato " + row['Campionato'],
        "phase": row['Girone'],
        "round": int(row['Giornata'].replace('.a','')),
        "home": row['Casa'],
        "opponents": row['Avversario'],
        "abbreviation": row['SiglaAvversario'],
        "referee1": str(row['Arbitro1']),
        "referee2": str(row['Arbitro2']),
        "location": str(row['Luogo']),
        "trainer": str(row['AvversariAllenatore'])
    }
    
    print(row['Giornata'] + ' ' + game['phase'] + ' - ' + game['opponents'])
    
    
    # Opponents info
    game['opponents_info'] = {}
    go = game['opponents_info']
        
    dfavv = avversari[avversari['IDpartita']==gameid]
    for i,r in dfavv.iterrows():
        go[r['Nome']] = {
            "name":   r['Nome'],
            "number": str(r['NumeroMaglia']),
            "year":   str(r['Anno']),
            "fouls":  int(r['Falli']),
            "points": int(r['Punti'])
        }
        
    game['unavailable_players'] = []
    
    if '2025' in game['date']:
        game['unavailable_players'].append('Dusels')
        game['unavailable_players'].append('Matteucci')
        

    
    # Game status
    game['status'] = {
        "quarter": row['Tempo'] - 1,
        "seconds": row['Secondi'],
        "gameover": True,
        "timeouts1": 0,
        "timeouts2": 0
    }
    
    if game['home']:
        game['status']['points1'] = row['Punti']
        game['status']['points2'] = row['AvversarioPunti']
        game['status']['fouls1']  = row['FCom']
        game['status']['fouls2']  = row['AvversarioFalli']
    else:
        game['status']['points1'] = row['AvversarioPunti']
        game['status']['points2'] = row['Punti']
        game['status']['fouls1']  = row['AvversarioFalli']
        game['status']['fouls2']  = row['FCom']
        

    # Convocati
    players = dict(zip(giocatori['ID'], giocatori['Cognome']))
    
    if not 'Perez Da Rold' in giocatori['Cognome']:
        game['unavailable_players'].append('Perez Da Rold')
    
    plusminus = {player: 0 for player in giocatori['Cognome']}

    game['on_field'] = []
        
    # Events
    events = []
    df = eventi[eventi['IDpartita']==gameid]
    for i,r in df.iterrows():
        evid        = int(r['IDevento'])
        idgiocatore = int(r['IDgiocatore'])
        tempo       = int(r['Tempo'])
        secondi     = 600.0 - float(r['Secondi'])
        x           = float(r['X'])
        y           = float(r['Y'])
        
        if idgiocatore < 0:
            team   = 'Opponents'
            x = int(r['X'])
            if x in list(dfavv['ID']):
                player = list(dfavv[dfavv['ID']==x]['Nome'])[0]
            else:
                #player = list(dfavv['Nome'])[0]
                player = ''

            punti = 0
            if evid == 0:    # Tiro libero segnato
                punti = 1
            elif evid == 2:  # Tiro da due segnato
                punti = 2
            elif evid == 4:  # Tiro da tre segnato
                punti = 3
                
            if punti > 0:
                for p in game['on_field']:
                    plusminus[p] -= punti
                
        elif idgiocatore == 0:
            team   = 'Team'
            player = 'Team'
        else:
            team   = 'Team'
            player = players[idgiocatore]

            punti = 0
            
            if evid == 18:
                if not player in game['on_field']:
                    game['on_field'].append(player)
            elif evid == 19:
                if player in game['on_field']:
                    game['on_field'].remove(player)
            elif evid == 0:  # Tiro libero segnato
                punti = 1
            elif evid == 2:  # Tiro da due segnato
                punti = 2
            elif evid == 4:  # Tiro da tre segnato
                punti = 3
                
            if punti > 0:
                for p in game['on_field']:
                    plusminus[p] += punti
                
    
        ev = {
            "team": team,
            "player": player,
            "event": evid,
            "event_name": EVENT_NAME[evid],
            "event_description": EVENT_DESCRIPTION[evid],
            "quarter": tempo,
            "seconds": secondi,
            "x": x,
            "y": y,
            "time": "2025-02-09 17:11:58"
        }
        events.append(ev)
        
    game['events'] = events
        
        

    # Players info
    game['players_info'] = {}

    df = convocati[convocati['IDpartita']==gameid]
    for i,r in df.iterrows():
        idgiocatore = int(r['IDgiocatore'])
        player = players[idgiocatore]
        
        game['players_info'][player] = {
            "name": player,
            "number": str(list(giocatori[giocatori['ID']==idgiocatore]['NumeroMaglia'])[0]),
            "time_on_field": float(r['Minuti']),
            "plusminus": plusminus[player]
        }
        
        
    filename = '../data/%d.a-%s-%s.game' % (game['round'], game['phase'], game['opponents'])
    with open(filename, 'w') as file:
        txt = json.dumps(game, indent=4, sort_keys=False)
        file.write(txt)
    
    #display(game)